# COVID-19 cases among healthcare workers

#### Updated 4/11/2020

### Load python tools

In [1]:
import pandas as pd
from shapely.geometry import shape
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import openpyxl
import altair as alt
from altair_saver import save
import lxml
import requests
from shapely.geometry import Point, LineString
import geojson
import json
import glob
import io
import os
import pyarrow
from shapely.geometry import Point, LineString, MultiLineString
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

### Read healthcare worker infections data from CHHS

In [2]:
hcworkers = pd.read_excel('input/HCW tables for LAT_20200408.xlsx')

In [3]:
hcworkers['County'] = hcworkers['County'].str.title()

In [4]:
hcworkers.columns = hcworkers.columns.str.lower()

In [5]:
hcworkers['healthcareworker'] = hcworkers['healthcareworker']\
    .astype(str).apply(lambda x: x.replace('<',''))

In [6]:
hcworkers['healthcareworker'] = hcworkers['healthcareworker'].astype(float)

### Read data from hospital utilization report

In [7]:
pop = pd.read_csv('/Users/mhustiles/data/github/notebooks/coronavirus/output/icus_county_pop_slim.csv')

In [8]:
pop = pop.drop(['id'], axis=1)

In [9]:
pop.head()

,county,total_beds,icu_beds,population,icus_per1000
0,Alameda,3746.0,264.0,1643700.0,16.061325
1,Amador,52.0,6.0,37829.0,15.860847
2,Butte,653.0,59.0,227075.0,25.982605
3,Calaveras,25.0,8.0,45235.0,17.685421
4,Colusa,48.0,6.0,21464.0,27.953783


---

### Read data from *LA Times* coronavirus tracker

In [10]:
counties = pd.read_json('/Users/mhustiles/data/github/coronavirus-tracker/_data/counties/totals/all.json')

In [11]:
testing = pd.read_json('/Users/mhustiles/data/github/coronavirus-tracker/_data/testing/timeseries.json')

In [12]:
counties['fips'] = counties['fips'].astype(str).str.zfill(3)

In [13]:
counties.head()

,county,confirmed_cases,deaths,new_confirmed_cases_today,new_deaths_today,new_confirmed_cases_seven_day_average,new_deaths_seven_day_average,agencies_count,agencies_updated,updated,fips,new_confirmed_cases_yesterday,new_deaths_yesterday
0,Alameda,964,36,77.0,13.0,41.428571,2.857143,2,2,True,001,-1,0
1,Alpine,1,0,NaN,NaN,0.000000,0.000000,1,0,False,003,0,0
2,Amador,7,0,NaN,NaN,0.571429,0.000000,1,0,False,005,0,0
3,Butte,16,0,NaN,NaN,0.428571,0.000000,1,0,False,007,1,0
4,Calaveras,11,0,2.0,0.0,0.857143,0.000000,1,1,True,009,0,0


---

### CA county geography

In [14]:
county_geo = gpd.read_file('/Users/mhustiles/data/data/gis/ca-counties.geojson')

In [15]:
county_geo.columns = county_geo.columns.str.lower()

In [16]:
len(county_geo)

58

---

### Merge with other dataframes

In [17]:
county_geo_pop = county_geo.merge(pop, left_on='name', right_on='county', how='left')

In [18]:
county_geo_pop_covid = county_geo_pop.merge(counties, on='county', how='left')

In [19]:
county_geo_pop_covid_all = county_geo_pop_covid.merge(hcworkers, on='county', how='left')

In [20]:
county_geo_pop_covid_all.to_file('output/county_geo_pop_covid_all.geojson', driver='GeoJSON')

---

In [21]:
workers = gpd.GeoDataFrame(county_geo_pop_covid_all[['fips_x', 'name', 'total_beds', \
                                    'population', 'confirmed_cases', \
                                    'healthcareworker', 'deaths', 'geometry']])

In [22]:
workers['hc_cases_share'] = ((workers.healthcareworker / workers.confirmed_cases)*100).round(2)
workers['hc_per_100k_pop'] = ((workers.healthcareworker * 100000)/workers.population).round(2)
workers['hc_per_100_hosp_beds'] = ((workers.healthcareworker * 100)/workers.total_beds).round(2)

In [23]:
workers['longitude'] = workers.centroid.map(lambda p: p.x)
workers['latitude'] = workers.centroid.map(lambda p: p.y)

### Statewide share of confirmed COVID cases that are healthcare workers

In [24]:
'{:,.2f}%'.format((workers.healthcareworker.sum() / workers.confirmed_cases.sum()*100))

'6.79%'

In [25]:
workers.drop(['geometry'], axis=1, inplace=True)

In [26]:
workers.rename(columns={'fips_x': 'fips'}, inplace=True)

In [27]:
workers.to_csv('output/workers.csv')

In [28]:
workersslim = workers[['fips','name','healthcareworker','hc_per_100k_pop',\
         'hc_per_100_hosp_beds','hc_per_100_hosp_beds', 'longitude','latitude']]

In [29]:
workersslim.to_csv('output/workersslim.csv', index=False)

In [30]:
hc_cases_share = workers[workers['healthcareworker'] > 5].sort_values(by='hc_cases_share', ascending=False)

In [31]:
hc_cases_share.to_csv('output/hc_cases_share.csv', index=False)

In [32]:
hc_per_100k_pop = workers[workers['healthcareworker'] > 5].sort_values(by='hc_per_100k_pop', ascending=False)

In [33]:
hc_per_100k_pop.to_csv('output/hc_per_100k_pop.csv', index=False)

In [34]:
hc_per_100_hosp_beds = workers[workers['healthcareworker'] > 5]\
.sort_values(by='hc_per_100_hosp_beds', ascending=False)

In [35]:
hc_per_100_hosp_beds.to_csv('output/hc_per_100_hosp_beds.csv', index=False)